In [1]:
import os
import perceval
import requests
import urllib
from bs4 import BeautifulSoup as bs
import pandas as pd 
import sys
import cv2
import os
import tqdm

# Scrapping the mail archives

In [2]:
url_mail_arc = 'https://mail-archives.apache.org/mod_mbox/'
host_mail_arc = 'mail-archives.apache.org'
base = pd.DataFrame(columns = {'project','mail_list_type','href'})
year = pd.DataFrame(columns = {'project','mail_list_type','href','monyear','mbox_link','total_mail'})
year2 = pd.DataFrame(columns = {'project','mail_list_type','href','monyear','mbox_link','total_mail'})
user = pd.DataFrame(columns = {'project','mail_list_type','href','monyear','mbox_link','total_mail','user'})

In [3]:
payload={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en;q=0.9",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Host": host_mail_arc,
        "If-Modified-Since": "Tue, 02 Nov 2021 00:57:28 GMT",
        "If-None-Match": "1dc43-5cfc3c708b690-gzip",
        "Referer": "https://mail-archives.apache.org/",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Sec-GPC": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
        }

In [4]:
s = requests.Session()
r = s.get(url_mail_arc,headers=payload)
soup = bs(r.text)
li = soup.findAll("td")
for i in tqdm.tqdm(li):
    mainli = i.findAll("ul",recursive=False)[0].findAll("li",recursive=False)
    for j in mainli:
        title = j.findAll('h3')[0].text
        for k in j.findAll("li"):
            base = base.append({'project':title,'mail_list_type':k.text, 'href':k.findAll('a')[0]['href']},ignore_index=True)

100%|██████████| 3/3 [00:01<00:00,  1.72it/s]


In [5]:
rachit = pd.read_csv('op.csv')

In [6]:
rachit = rachit[rachit['Is Source?']==True].reset_index(drop=True)

In [7]:
rachit['repo'] = rachit['Repo Name'].str.split('/').str[-1]

In [8]:
year = pd.read_csv('mail_archive_links.csv').drop({'Unnamed: 0'},axis=1)

In [9]:
for _,i in rachit.iterrows():
    year2 = year2.append(year[year['href'].str.contains(i.repo)],
                         ignore_index=True)

In [21]:
for _,i in tqdm.tqdm(year2[1000:5000].iterrows()):
    s = requests.Session()
    r = s.get(url_mail_arc+i.href+i.mbox_link+'/ajax/thread?0',headers=payload)
    soup = bs(r.text,'xml')
    #print(url_mail_arc+i.href+i.mbox_link+'/ajax/thread?0')
    for j in soup.findAll('from'):
        user = user.append({'project':i.project,'mail_list_type':i.mail_list_type,
                        'href':i.href,'monyear':i.monyear,
                        'mbox_link':i.mbox_link,
                        'total_mail':i.total_mail,'user':j.string},ignore_index=True)

4000it [7:23:15,  6.65s/it] 


In [22]:
user.to_csv('mid_user_4.csv')

In [23]:
user

,mbox_link,monyear,user,href,total_mail,project,mail_list_type
0,201105.mbox,May 2011,br...@apache.org,maven-wagon-commits/,22,maven.apache.org lists:,wagon-commits
1,201105.mbox,May 2011,denn...@apache.org,maven-wagon-commits/,22,maven.apache.org lists:,wagon-commits
2,201105.mbox,May 2011,strub...@apache.org,maven-wagon-commits/,22,maven.apache.org lists:,wagon-commits
3,201105.mbox,May 2011,strub...@apache.org,maven-wagon-commits/,22,maven.apache.org lists:,wagon-commits
4,201105.mbox,May 2011,strub...@apache.org,maven-wagon-commits/,22,maven.apache.org lists:,wagon-commits
...,...,...,...,...,...,...,...
698243,201306.mbox,Jun 2013,jo...@apache.org,opennlp-commits/,10,opennlp.apache.org lists:,commits
698244,201306.mbox,Jun 2013,auta...@apache.org,opennlp-commits/,10,opennlp.apache.org lists:,commits
698245,201306.mbox,Jun 2013,auta...@apache.org,opennlp-commits/,10,opennlp.apache.org lists:,commits
698246,201306.mbox,Jun 2013,build...@apache.org,opennlp-commits/,10,opennlp.apache.org lists:,commits


In [16]:
user['mail_list_type'].unique()

array(['wagon-commits', 'wagon-dev', 'wagon-users', 'fop-commits',
       'fop-dev', 'fop-users', 'scm-commits', 'scm-dev', 'scm-users',
       'pluto-dev', 'pluto-scm', 'pluto-user', 'commits', 'dev',
       'general', 'user', 'doxia-commits', 'doxia-dev', 'doxia-users',
       'scm', 'xbean-dev', 'xbean-scm', 'xbean-user', 'site-pub',
       'distributedlog-commits', 'distributedlog-dev',
       'distributedlog-issues', 'distributedlog-user', 'issues',
       'rampart-c-dev', 'rampart-dev', 'notifications', 'users',
       'oak-commits', 'oak-dev', 'oak-issues'], dtype=object)

In [ ]:
os.chdir('/Users/suvivarshney/Documents/UCD_Live/ECS260 SE/Project/mail_arc/')
for _,i in tqdm.tqdm(year.iterrows()):
    os.system('wget '+url_mail_arc+i.href+i.mbox_link)

34365it [9:18:31,  1.19s/it] 

In [ ]:
year

# Scrapping the repo status and other info 

In [22]:
url_status = 'https://projects.apache.org/projects.html?name'
host_mail_arc = 'projects.apache.org'
podling_url = 'https://projects.apache.org/json/foundation/podlings.json'
podling_history_url = 'https://projects.apache.org/json/foundation/podlings-history.json'
comittee_retired_url = 'https://projects.apache.org/json/foundation/committees-retired.json'

In [137]:
payload={
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
        "Accept-Encoding": "gzip, deflate, br",
        "Accept-Language": "en-GB,en;q=0.9",
        "Cache-Control": "max-age=0",
        "Connection": "keep-alive",
        "Host": host_mail_arc,
        "If-Modified-Since": "Tue, 02 Nov 2021 00:57:28 GMT",
        "If-None-Match": "1dc43-5cfc3c708b690-gzip",
        "Referer": "https://mail-archives.apache.org/",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "same-origin",
        "Sec-Fetch-User": "?1",
        "Sec-GPC": "1",
        "Upgrade-Insecure-Requests": "1",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"
        }

In [18]:
urllib.request.urlretrieve(podling_url, "podling.json")

('podling.json', <http.client.HTTPMessage at 0x7fcc8943f9d0>)

In [20]:
urllib.request.urlretrieve(podling_history_url, "podling_history.json ")

('podling_history.json ', <http.client.HTTPMessage at 0x7fcc3802d460>)

In [23]:
urllib.request.urlretrieve(comittee_retired_url, "comittee_retired_url.json ")

('comittee_retired_url.json ', <http.client.HTTPMessage at 0x7fcc3802d0a0>)